# Tool Calling 101:

This is part (1/2) in the tool calling series, this notebook will cover the basics of what tool calling is and how to perform it with `Llama 3.1 models`

Here's what you will learn in this notebook:

- Setup Groq to access Llama 3.1 70B model
- Avoid common mistakes when performing tool-calling with Llama
- Understand Prompt templates for Tool Calling
- Understand how the tool calls are handled under the hood

In Part 2, we will learn how to build system that can get us comparision between 2 papers

## What is Tool Calling?

This approach was popularised by the [Gorilla](https://gorilla.cs.berkeley.edu) paper-which showed that Large Language Model(s) can be fine-tuned on API examples to teach them calling an external API. 

This is really cool because we can now use a LLM as a "brain" of a system and connect it to external systems to perform actions. 

In simpler words, "Llama can order your pizza for you" :) 

With the Llama 3.1 release, the models excel at tool calling and support out of box `brave_search`, `wolfram_api` and `code_interpreter`. 

However, first let's take a look at a common mistake

#### Install and setup groq dependencies

- Install `groq` api to access Llama model(s)
- Configure our client and authenticate with API Key(s)

In [1]:
#!pip3 install groq

In [82]:
import os
from groq import Groq
# Create the Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"), )

## Common Mistake of Tool-Calling: Incorrect Prompt Template

While Llama 3.1 works with tool-calling out of box, a wrong prompt template can cause issues with unexpected behaviour. 

Sometimes, even superheroes need to be reminded of their powers. 

Let's first try "forcing a prompt response from the model"

#### Note: Remember this is the WRONG template, please scroll to next section to see the right approach if you are in a rushed copy-pasta sprint

This section will show you that the model will not use `brave_search` and `wolfram_api` out of the box unless the prompt template is set correctly. 
Even if the model is asked to do so!

In [83]:
SYSTEM_PROMPT = """
Cutting Knowledge Date: December 2023
Today Date: 20 August 2024

You are a helpful assistant
"""

In [84]:
system_prompt = {}
chat_history = []

def model_chat(user_input: str, sys_prompt = SYSTEM_PROMPT, temperature: int = 0.7, max_tokens=2048):
    
    chat_history = [
        {
            "role": "system",
            "content": sys_prompt
        }
    ]
    
    chat_history.append({"role": "user", "content": user_input})
    
    #print(chat_history)
    
    #print("User: ", user_input)
    
    response = client.chat.completions.create(model="llama-3.1-70b-versatile",
                                          messages=chat_history,
                                          max_tokens=max_tokens,
                                          temperature=temperature)
    
    chat_history.append({
    "role": "assistant",
    "content": response.choices[0].message.content
    })
    
    
    #print("Assistant:", response.choices[0].message.content)
    
    return response.choices[0].message.content

#### Asking the model about a recent news

Since the prompt template is incorrect, it will answer using cutoff memory

In [85]:
user_input = """
When is the next elden ring game coming out?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))

Assistant: Unfortunately, I don't have information on a specific release date for the next Elden Ring game. However, I can tell you that there have been rumors and speculations about a potential sequel or DLC (Downloadable Content) for Elden Ring.

In June 2022, the game's director, Hidetaka Miyazaki, mentioned that FromSoftware, the developer of Elden Ring, was working on "multiple" new projects, but no official announcements have been made since then.

It's also worth noting that FromSoftware has a history of taking their time to develop new games, and the studio is known for its attention to detail and commitment to quality. So, even if there is a new Elden Ring game in development, it's likely that we won't see it anytime soon.

Keep an eye on official announcements from FromSoftware and Bandai Namco, the publisher of Elden Ring, for any updates on a potential sequel or new game in the series.


#### Asking the model about a Math problem

Again, the model answer(s) based on memory and not tool-calling

In [86]:
user_input = """
When is the square root of 23131231?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))


Assistant: To find the square root of 23131231, I'll calculate it for you.

√23131231 ≈ 4813.61


#### Can we solve this using a reminder prompt?

In [87]:
user_input = """
When is the square root of 23131231?

Can you use a tool to solve the question?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))


Assistant: I can use a mathematical tool to solve the question.

The square root of 23131231 is:

√23131231 ≈ 4810.51


Looks like we didn't get the wolfram_api call, let's try one more time with a stronger prompt:

In [88]:
user_input = """
When is the square root of 23131231?

Can you use a tool to solve the question?

Remember you have been trained on wolfram_alpha
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))


Assistant: I can use Wolfram Alpha to calculate the square root of 23131231.

According to Wolfram Alpha, the square root of 23131231 is:

√23131231 ≈ 4809.07


### Official Prompt Template 

As you can see, the model doesn't perform tool-calling in an expected fashion above. This is because we are not following the recommended prompting format.

The Llama Stack is the go to approach to use the Llama model family and build applications. 

Let's first install the `llama_toolchain` Python package to have the Llama CLI available.

In [2]:
#!pip3 install llama-toolchain

#### Now we can learn about the various prompt formats available 

When you run the cell below-you will see all the options for different template

In [90]:
!llama model template

+-----------+---------------------------------+
| Role      | Template Name                   |
+-----------+---------------------------------+
| user      | user-default                    |
| assistant | assistant-builtin-tool-call     |
| assistant | assistant-custom-tool-call      |
| assistant | assistant-default               |
| system    | system-builtin-and-custom-tools |
| system    | system-builtin-tools-only       |
| system    | system-custom-tools-only        |
| system    | system-default                  |
| tool      | tool-success                    |
| tool      | tool-failure                    |
+-----------+---------------------------------+


## Tool Calling: Using the correct Prompt Template

With `llama-cli` we can learn the correct way of defining `System_prompt` and finally get the correct behaviour from the model

In [92]:
!llama model template --name system-builtin-tools-only

+----------+--------------------------------------------------------------+
| Name     | system-builtin-tools-only                                    |
+----------+--------------------------------------------------------------+
| Template | <|begin_of_text|><|start_header_id|>system<|end_header_id|>↵ |
|          | ↵                                                            |
|          | Environment: ipython↵                                        |
|          | Tools: brave_search, wolfram_alpha↵                          |
|          | Cutting Knowledge Date: December 2023↵                       |
|          | Today Date: 15 September 2024↵                               |
|          | ↵                                                            |
|          | You are a helpful assistant.↵                                |
|          | <|eot_id|><|start_header_id|>assistant<|end_header_id|>↵     |
|          | ↵                                                            |
|          |

If everything is setup correctly-the model should now wrap function calls  with the `|<python_tag>|` following the actualy function call. 

This can allow you to manage your function calling logic accordingly. 

Time to test the theory

In [94]:
!llama model template --name assistant-builtin-tool-call

+----------+----------------------------------------------------------------------------------+
| Name     | assistant-builtin-tool-call                                                      |
+----------+----------------------------------------------------------------------------------+
| Template | <|begin_of_text|><|start_header_id|>assistant<|end_header_id|>↵                  |
|          | ↵                                                                                |
|          | <|python_tag|>brave_search.call(query="Who won NBA in                            |
|          | 2024?")<|eom_id|><|start_header_id|>assistant<|end_header_id|>↵                  |
|          | ↵                                                                                |
|          |                                                                                  |
+----------+----------------------------------------------------------------------------------+
| Notes    | ↵ represents newline       

In [95]:
SYSTEM_PROMPT = """
Environment: iPython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 15 September 2024
"""

user_input = """
When is the next Elden ring game coming out?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))


Assistant: <|python_tag|>brave_search.call(query="Elden Ring sequel release date")


In [96]:
user_input = """
What is the square root of 23131231?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))

Assistant: <|python_tag|>wolfram_alpha.call(query="square root of 23131231")


### Using this knowledge in practise

A common misconception about tool calling is: the model can handle the tool call and get your output. 

This is NOT TRUE, the actual tool call is something that you have to implement. With this knowledge, let's see how we can utilise brave search to answer our original question

In [97]:
#!pip3 install brave-search

In [98]:
SYSTEM_PROMPT = """
Environment: iPython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 15 September 2024
"""

user_input = """
What is the square root of 23131231?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))

Assistant: <|python_tag|>wolfram_alpha.call(query="square root of 23131231")


In [99]:
print(model_chat(user_input, sys_prompt=SYSTEM_PROMPT))

output = model_chat(user_input, sys_prompt=SYSTEM_PROMPT)

<|python_tag|>wolfram_alpha.call(query="square root of 23131231")


In [102]:
import re

# Extract the function name
fn_name = re.search(r'<\|python_tag\|>(\w+)\.', output).group(1)

# Extract the method
fn_call_method = re.search(r'\.(\w+)\(', string).group(1)

# Extract the arguments
fn_call_args = re.search(r'=\s*([^)]+)', string).group(1)

print(f"Function name: {fn_name}")
print(f"Method: {fn_call_method}")
print(f"Args: {fn_call_args}")

Function name: wolfram_alpha
Method: call
Args: "square root of 23131231"


You can implement this in different ways but the idea is the same, the LLM gives an output with the `<|python_tag|>`, which should call a tool-calling mechanism. 

This logic gets handled in the program and then the output is passed back to the model to answer the user

### Code interpreter

With the correct prompt template, Llama model can output Python (as well as code in any-language that the model has been trained on)

In [54]:
user_input = """

If I can invest 400$ every month at 5% interest rate, how long would it take me to make a 100k$ in investments?
"""

print("Assistant:", model_chat(user_input, sys_prompt=SYSTEM_PROMPT))

Assistant: <|python_tag|>import math

# Define the variables
monthly_investment = 400
interest_rate = 0.05
target_amount = 100000

# Calculate the number of months it would take to reach the target amount
months = 0
current_amount = 0
while current_amount < target_amount:
    current_amount += monthly_investment
    current_amount *= 1 + interest_rate / 12  # Compound interest
    months += 1

# Print the result
print(f"It would take {months} months, approximately {months / 12:.2f} years, to reach the target amount of ${target_amount:.2f}.")


Let's validate the output by running the output from the model:

In [55]:
# Define the variables
monthly_investment = 400
interest_rate = 0.05
target_amount = 100000

# Calculate the number of months it would take to reach the target amount
months = 0
current_amount = 0
while current_amount < target_amount:
    current_amount += monthly_investment
    current_amount *= 1 + interest_rate / 12  # Compound interest
    months += 1

# Print the result
print(f"It would take {months} months, approximately {months / 12:.2f} years, to reach the target amount of ${target_amount:.2f}.")

It would take 172 months, approximately 14.33 years, to reach the target amount of $100000.00.


In [56]:
#fin